In [1]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
NUM_CAMERAS = 10

In [3]:
# setup demo environment with 10 cameras
os.environ["PYLON_CAMEMU"] = f"{NUM_CAMERAS}"

In [4]:
tlf = py.TlFactory.GetInstance()

In [5]:
# create a device filter for Pylon CamEmu devices
di = py.DeviceInfo()
di.SetDeviceClass("BaslerCamEmu")

# you could also set more device filters like e.g.:
# these are combined as AND 
# di.SetSerialNumber("2134234")

<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163E1CD7B40> >

In [6]:
devs = tlf.EnumerateDevices([di,])

In [7]:
devs

(<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FB83F660> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FC3E7F60> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163E1CC8AE0> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163E1CD7360> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163E1CD7660> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FC401030> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FC401180> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FC4010F0> >,
 <pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x00000163FC401300> >,
 <pypylon.pylon.DeviceInfo; proxy of 

In [8]:
cam_array = py.InstantCameraArray(NUM_CAMERAS)

In [9]:
for idx, cam in enumerate(cam_array):
    cam.Attach(tlf.CreateDevice(devs[idx]))

In [10]:
cam_array.Open()

In [11]:
# store a unique number for each camera to identify the incoming images
for idx, cam in enumerate(cam_array):
    camera_serial = cam.DeviceInfo.GetSerialNumber()
    print(f"set context {idx} for camera {camera_serial}")
    cam.SetCameraContext(idx)

set context 0 for camera 0815-0000
set context 1 for camera 0815-0001
set context 2 for camera 0815-0002
set context 3 for camera 0815-0003
set context 4 for camera 0815-0004
set context 5 for camera 0815-0005
set context 6 for camera 0815-0006
set context 7 for camera 0815-0007
set context 8 for camera 0815-0008
set context 9 for camera 0815-0009


In [12]:
# set the exposure time for each camera
for idx, cam in enumerate(cam_array):
    camera_serial = cam.DeviceInfo.GetSerialNumber()
    print(f"set Exposuretime {idx} for camera {camera_serial}")
    cam.ExposureTimeRaw = 10000

set Exposuretime 0 for camera 0815-0000
set Exposuretime 1 for camera 0815-0001
set Exposuretime 2 for camera 0815-0002
set Exposuretime 3 for camera 0815-0003
set Exposuretime 4 for camera 0815-0004
set Exposuretime 5 for camera 0815-0005
set Exposuretime 6 for camera 0815-0006
set Exposuretime 7 for camera 0815-0007
set Exposuretime 8 for camera 0815-0008
set Exposuretime 9 for camera 0815-0009


In [13]:
# wait for all cameras to grab 10 frames
frames_to_grab = 10
# store last framecount in array
frame_counts = [0]*NUM_CAMERAS

In [14]:
frame_counts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
cam_array.StartGrabbing()
while True:
    with cam_array.RetrieveResult(1000) as res:
        if res.GrabSucceeded():
            img_nr = res.ImageNumber
            cam_id = res.GetCameraContext()
            frame_counts[cam_id] = img_nr
            print(f"cam #{cam_id}  image #{img_nr}")
            
            # do something with the image ....
            
            # check if all cameras have reached 100 images
            if min(frame_counts) >= frames_to_grab:
                print( f"all cameras have acquired {frames_to_grab} frames")
                break
                
                
cam_array.StopGrabbing()

cam #5  image #1
cam #0  image #1
cam #1  image #1
cam #2  image #1
cam #3  image #1
cam #4  image #1
cam #6  image #1
cam #7  image #1
cam #8  image #1
cam #9  image #1
cam #7  image #2
cam #2  image #2
cam #0  image #2
cam #5  image #2
cam #4  image #2
cam #9  image #2
cam #1  image #2
cam #8  image #2
cam #6  image #2
cam #3  image #2
cam #3  image #3
cam #2  image #3
cam #6  image #3
cam #1  image #3
cam #4  image #3
cam #0  image #3
cam #8  image #3
cam #5  image #3
cam #9  image #3
cam #7  image #3
cam #1  image #4
cam #7  image #4
cam #3  image #4
cam #0  image #4
cam #8  image #4
cam #9  image #4
cam #5  image #4
cam #2  image #4
cam #6  image #4
cam #4  image #4
cam #5  image #5
cam #0  image #5
cam #7  image #5
cam #3  image #5
cam #2  image #5
cam #8  image #5
cam #6  image #5
cam #9  image #5
cam #4  image #5
cam #1  image #5
cam #6  image #6
cam #9  image #6
cam #8  image #6
cam #1  image #6
cam #2  image #6
cam #4  image #6
cam #5  image #6
cam #3  image #6
cam #7  image 

In [16]:
cam_array.Close()

In [17]:
frame_counts

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]